In [61]:
# %%
import torch
from torch import nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
from tqdm import tqdm, trange

In [62]:

# Détection du périphérique (GPU ou CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Prétraitement
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225]),
])

# Données
train_data = datasets.ImageFolder("data/train", transform=transform)
val_data = datasets.ImageFolder("data/val", transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False, num_workers=4)

# Nombre de classes
num_classes = len(train_data.classes)
print("Classes :", train_data.classes)

# %%


Classes : ['Lynx', 'Salamandre', 'Tortue']


In [63]:
#train_data_samples = next(iter(train_loader))

# train_data_samples

In [64]:
# Définition du CNN
class NeuralNetwork(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)

        self.fc1 = nn.Linear(128 * 16 * 16, 256)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # [32x64x64]
        x = self.pool(F.relu(self.conv2(x)))  # [64x32x32]
        x = self.pool(F.relu(self.conv3(x)))  # [128x16x16]
        x = x.view(x.size(0), -1)             # flatten
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.2, training=self.training)  # Dropout pour éviter le surapprentissage
        x = F.relu(self.fc2(x))
        x = F.dropout(x, p=0.2, training=self.training)  # Dropout pour éviter le surapprentissages
        x = self.fc3(x)
        return x

model = NeuralNetwork(num_classes).to(device)


In [65]:
# %%
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# %%


In [66]:
def train():
    print("Training started")
    model.train()
    running_loss = 0
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        preds = model(images)
        loss = loss_fn(preds, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Train Loss: {running_loss/len(train_loader):.4f}")


In [67]:
def validate():
    print("Validating...")
    model.eval()
    total, correct = 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            preds = model(images)
            correct += (preds.argmax(1) == labels).sum().item()
            total += labels.size(0)
    acc = 100 * correct / total
    print(f"Validation Accuracy: {acc:.2f}%")

# %%


In [68]:
# Boucle d'entraînement
for epoch in trange(5, desc="Epochs"):
    print(f"\nEpoch {epoch+1}")
    train()
    validate()

Epochs:   0%|          | 0/5 [00:00<?, ?it/s]


Epoch 1
Training started
Train Loss: 0.0549
Validating...


Epochs:  20%|██        | 1/5 [05:30<22:01, 330.37s/it]

Validation Accuracy: 99.16%

Epoch 2
Training started
Train Loss: 0.0236
Validating...


Epochs:  40%|████      | 2/5 [15:08<23:49, 476.36s/it]

...

Validation Accuracy: 99.81%

Epoch 4
Training started
Train Loss: 0.0237
Validating...


Epochs:  80%|████████  | 4/5 [26:22<06:21, 381.57s/it]

Validation Accuracy: 99.72%

Epoch 5
Training started
Train Loss: 0.0193
Validating...


Epochs: 100%|██████████| 5/5 [31:55<00:00, 383.02s/it]

Validation Accuracy: 98.92%


In [69]:
# %%
torch.save(model.state_dict(), "animal_cnn.pth")
with open("animal_classes.txt", "w") as f:
    f.write("\n".join(train_data.classes))
print("Modèle sauvegardé.")


Modèle sauvegardé.


In [70]:
# from collections import Counter

# # Liste des classes
# print("Classes :", train_data.classes)

# # Répartition des images dans le dossier
# labels = [label for _, label in train_data]
# counter = Counter(labels)
# print("Distribution des classes (train) :")
# for idx, count in counter.items():
#     print(f"{train_data.classes[idx]}: {count}")

# val_labels = [label for _, label in val_data]
# val_counter = Counter(val_labels)
# print("Distribution des classes (val) :")
# for idx, count in val_counter.items():
#     print(f"{val_data.classes[idx]}: {count}")



In [71]:
import onnxruntime as ort
import numpy as np

# exemple input numpy
input_np = np.random.rand(1, 3, 128, 128).astype(np.float32)

sess = ort.InferenceSession("my_model.onnx")
inputs = {sess.get_inputs()[0].name: input_np}
outputs = sess.run(None, inputs)
print(outputs)


[array([[-0.05675411, -0.01766702, -0.041874  ]], dtype=float32)]


In [ ]:

model.eval()

# Define dummy input with batch size 1 and correct image size 3x128x128
tensor_torch = torch.randn(1, 3, 128, 128)

# Export the model
torch.onnx.export(
    model,
    tensor_torch,
    "my_model.onnx",
    input_names = ['input'],
    output_names = ['output']
)



In [ ]:
# DEBUG: Test de l'inférence avec le modèle PyTorch et ONNX

from PIL import Image
import torch
from torchvision import transforms
import onnxruntime as ort
import numpy as np

# Charger ton image de test (la même que sur ton site)
image_path = "save/Lynx/_test.jpg"
image = Image.open(image_path).convert("RGB")  # assure 3 canaux

# Appliquer les mêmes transformations que pendant l'entraînement
transform = transforms.Compose([
    transforms.Resize((128, 128)),                 # même taille
    transforms.ToTensor(),                         # [0, 1]
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # même normalisation
                         std=[0.229, 0.224, 0.225])
])
tensor_image = transform(image).unsqueeze(0)  # shape (1, 3, 128, 128)

# Faire passer l'image dans ton modèle PyTorch
model.eval()
with torch.no_grad():
    pytorch_output = model(tensor_image)
    print("PyTorch output:", pytorch_output)

#  Charger le modèle ONNX
ort_sess = ort.InferenceSession("my_model.onnx")

# Transformer le tensor en numpy pour ONNX
input_numpy = tensor_image.numpy().astype(np.float32)

# Lancer l'inférence
outputs = ort_sess.run(None, {ort_sess.get_inputs()[0].name: input_numpy})

print("ONNX output:", outputs[0])



PyTorch output: tensor([[ 6.4333, -6.2365, -0.4407]])
ONNX output: [[ 6.433262   -6.2365146  -0.44074833]]
